## A Slightly Annotated Version of the Example Notebook

In [1]:
from math import log, sqrt

In [2]:
import numpy as np
import pandas as pd
from six.moves import cStringIO as StringIO

## six is a package the facilitates python 2.x to python 3.x compatibillity

In [3]:
from bokeh.plotting import *

I wish they wouldn't do they import \*. Makes it harder to figure out details.

In [4]:
antibiotics = """
bacteria,                        penicillin, streptomycin, neomycin, gram
Mycobacterium tuberculosis,      800,        5,            2,        negative
Salmonella schottmuelleri,       10,         0.8,          0.09,     negative
Proteus vulgaris,                3,          0.1,          0.1,      negative
Klebsiella pneumoniae,           850,        1.2,          1,        negative
Brucella abortus,                1,          2,            0.02,     negative
Pseudomonas aeruginosa,          850,        2,            0.4,      negative
Escherichia coli,                100,        0.4,          0.1,      negative
Salmonella (Eberthella) typhosa, 1,          0.4,          0.008,    negative
Aerobacter aerogenes,            870,        1,            1.6,      negative
Brucella antracis,               0.001,      0.01,         0.007,    positive
Streptococcus fecalis,           1,          1,            0.1,      positive
Staphylococcus aureus,           0.03,       0.03,         0.001,    positive
Staphylococcus albus,            0.007,      0.1,          0.001,    positive
Streptococcus hemolyticus,       0.001,      14,           10,       positive
Streptococcus viridans,          0.005,      10,           40,       positive
Diplococcus pneumoniae,          0.005,      11,           10,       positive
"""

In [5]:
# RGB/RGBA tuples do not work
# RGBA hex colors also do not work
drug_color = {
    "Penicillin"   : "#0d3362", 
    "Streptomycin" : '#8000ff',
    "Neomycin"     : "black",
}

In [6]:
gram_color = {
    "positive" : "#aeaeb8",
    "negative" : "#e69584",
}

In [19]:
df = pd.read_csv(StringIO(antibiotics), skiprows=1, skipinitialspace=True, engine='python')

In [20]:
width = 800
height = 800
inner_radius = 90#190
outer_radius = 300 - 10

minr = sqrt(log(.001 * 1E4))
maxr = sqrt(log(1000 * 1E4))
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

x = np.zeros(len(df))
y = np.zeros(len(df))

In [21]:
def rad(mic):
    return a * np.sqrt(np.log(mic * 1E4)) + b

In [22]:
output_notebook()

Loading BokehJS ...

In [23]:
help(figure)

Help on function figure in module bokeh.plotting.figure:

figure(**kwargs)
    Create a new :class:`~bokeh.plotting.figure.Figure` for plotting.
    
    In addition to the standard :class:`~bokeh.plotting.figure.Figure`
    property values (e.g. ``plot_width`` or ``sizing_mode``) the following
    additional options can be passed as well:
    
    .. bokeh-options:: FigureOptions
        :module: bokeh.plotting.figure
    
    Returns:
       Figure



## Lots of kwargs here that are not explained with the help function

* Search the documentation or play

In [24]:
p = figure(plot_width=width, plot_height=height, title="This is a title",
           x_axis_type=None, y_axis_type=None, # By default y/x_axis_type will draw a numeric axis
           x_range=[-420, 420], y_range=[-420, 420],
           min_border=10, outline_line_color="indigo",
           background_fill_color="#f0e1d2", border_fill_color="#f0e1d2")

In [25]:
p.line(x+1, y+1)

GlyphRenderer(id='1beb57e9-66a8-4f5b-a1c0-ae45d275caf2', ...)

In [26]:
# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [gram_color[gram] for gram in df.gram]
p.annular_wedge(x, y, inner_radius, outer_radius, -big_angle+angles, angles, color=colors)

GlyphRenderer(id='de2fb95a-b4a4-49aa-b26f-06dd73f10963', ...)

In [27]:
# small wedges
p.annular_wedge(x, y, inner_radius, rad(df.penicillin), -big_angle+angles + 5*small_angle, 
                -big_angle+angles+6*small_angle, color=drug_color['Penicillin'])
p.annular_wedge(x, y, inner_radius, rad(df.streptomycin), -big_angle+angles + 3*small_angle, 
                -big_angle+angles+4*small_angle, color=drug_color['Streptomycin'])
p.annular_wedge(x, y, inner_radius, rad(df.neomycin), -big_angle+angles + 1*small_angle, 
                -big_angle+angles+2*small_angle, color=drug_color['Neomycin'])

GlyphRenderer(id='2dfd021d-2656-423a-92f2-61534f60861f', ...)

In [28]:
# circular axes and lables
labels = np.power(10.0, np.arange(-3, 4))
radii = a * np.sqrt(np.log(labels * 1E4)) + b
p.circle(x, y, radius=radii, fill_color=None, line_color="white")
p.text(x[:-1], radii[:-1], [str(r) for r in labels[:-1]], 
       text_font_size="8pt", text_align="center", text_baseline="middle")

/opt/conda/lib/python3.5/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/opt/conda/lib/python3.5/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/opt/conda/lib/python3.5/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))


GlyphRenderer(id='4d9be289-de48-4a38-9224-3d5fe7ec16ff', ...)

In [29]:
# radial axes
p.annular_wedge(x, y, inner_radius-10, outer_radius+10, 
                -big_angle+angles, -big_angle+angles, color="black")

GlyphRenderer(id='581d242d-66f0-4b19-b4b9-fc52aacd5e17', ...)

In [30]:
# bacteria labels
xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.bacteria, angle=label_angle, text_font_size="9pt", text_align="center", text_baseline="middle")

GlyphRenderer(id='de149b5a-ce03-47d8-9bd3-8d0f7fd95eb6', ...)

### Like the programmers comment says, pretty clunky hack of a legend

In [31]:
# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-40, -40], [-370, -390], color=list(gram_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["Gram-" + x for x in gram_color.keys()], 
       text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13, color=list(drug_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(drug_color.keys()),
       text_font_size="9pt", text_align="left", text_baseline="middle")

GlyphRenderer(id='2af4cf25-b75a-43c7-9e29-5c42200f6e37', ...)

In [32]:
p.xgrid.grid_line_color = 'red'
p.ygrid.grid_line_color = None#'green' # Doesn't matter what we do here because the axes have been set to None

In [33]:
show(p)